In [1]:
import dlib         
import numpy as np  
import cv2          
import time
import os
from math import hypot

# 储存截图的目录
path_screenshots = "D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/"

# Dlib 正向人脸检测
detector = dlib.get_frontal_face_detector()
# Dlib 人脸特征点预测
predictor = dlib.shape_predictor('data/dlib/shape_predictor_68_face_landmarks.dat')

# 创建 cv2 摄像头对象
cap = cv2.VideoCapture(0)

# cap.set(propId, value)
# 设置视频参数，propId 设置的视频参数，value 设置的参数值
cap.set(3,640) #设置分辨率
cap.set(4,480)


# Delete all the screenshots
def del_ss():
    ss = os.listdir("data/screenshots/")
    for image in ss:
        print("Remove: ", "data/screenshots/"+image)
        os.remove("data/screenshots/"+image)


# del_ss()

def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    #利用脸谱特征图上的点，获得人脸上眼睛两边的坐标

    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))
    #利用脸谱特征图上的点，获得人脸上眼睛上下眼皮的坐标，同时计算中间点的坐标

    #hor_line = cv2.line(im_rd, left_point, right_point, (0,255,0), 3)
    #ver_line = cv2.line(im_rd, center_top, center_bottom, (0,255,255), 3)
    #将眼睛左右与上下连成线，方便观测

    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    #利用hypot函数计算得出线段的长度

    ratio = hor_line_lenght / (ver_line_lenght+1)
    #得到长宽比
    return ratio

def get_yawning_ratio(mouth_points, facial_landmarks):#mouth
    topleft = (facial_landmarks.part(mouth_points[0]).x, facial_landmarks.part(mouth_points[0]).y)
    topright = (facial_landmarks.part(mouth_points[1]).x, facial_landmarks.part(mouth_points[1]).y)
    bottomleft = (facial_landmarks.part(mouth_points[2]).x, facial_landmarks.part(mouth_points[2]).y)
    bottomright = (facial_landmarks.part(mouth_points[3]).x, facial_landmarks.part(mouth_points[3]).y)
    
    width = (hypot((topright[0] - topleft[0]), (topright[1] - topleft[1]))+hypot((bottomright[0] - bottomleft[0]), (bottomright[1] - bottomleft[1])))/2
    height = (hypot((topright[0] - bottomright[0]), (topright[1] - bottomright[1]))+hypot((topleft[0] - bottomleft[0]), (topleft[1] - bottomleft[1])))/2
    #利用hypot函数计算得出线段的长度

    ratio = width / height
    #得到长宽比
    return ratio

def noseposition(nose_points, facial_landmarks):#nose&nod
    nosepos = (facial_landmarks.part(nose_points[0]).x, facial_landmarks.part(nose_points[0]).y)
    
    #pos = hypot(nosepos[0], nosepos[1])
    pos = hypot(0, nosepos[1])
    for num in range(0,9):
        pos_arr[num] = pos_arr[num+1]
        
    pos_arr[9] = pos
    d_err= np.var(pos_arr)
    
    
    
    #return pos
    return d_err

def drawframe(face_points, facial_landmarks, r, g, b):##0 8 16 19,24
    left_point = (facial_landmarks.part(face_points[0]).x, facial_landmarks.part(face_points[0]).y)
    right_point = (facial_landmarks.part(face_points[2]).x, facial_landmarks.part(face_points[2]).y)
    
    center_top = midpoint(facial_landmarks.part(face_points[3]), facial_landmarks.part(face_points[4]))
    center_bottom = (facial_landmarks.part(face_points[1]).x, facial_landmarks.part(face_points[1]).y)
    
    left_top = (left_point[0] - int(0.1*(right_point[0]-left_point[0])),center_top[1] - int(0.2*(center_bottom[1] - center_top[1])))
    left_bottom = (left_point[0] - int(0.1*(right_point[0]-left_point[0])),center_bottom[1] + int(0.05*(center_bottom[1] - center_top[1])))
    right_top = (right_point[0] + int(0.1*(right_point[0]-left_point[0])),center_top[1] - int(0.2*(center_bottom[1] - center_top[1])))
    right_bottom = (right_point[0] + int(0.1*(right_point[0]-left_point[0])),center_bottom[1] + int(0.05*(center_bottom[1] - center_top[1])))
    
    ver_line = cv2.line(im_rd, left_top, left_bottom, (r, g, b), 2)
    ver_line = cv2.line(im_rd, right_top, right_bottom, (r, g, b), 2)
    hor_line = cv2.line(im_rd, left_top, right_top, (r, g, b), 2)
    hor_line = cv2.line(im_rd, left_bottom, right_bottom, (r, g, b), 2)

    return 


ss_cnt = 0# 截图 screenshots 的计数器

pos_arr = np.arange(10)
eye_arr = np.arange(15)

start1 = np.arange(10)#时间1 用于处理眼睛
start2 = np.arange(10)#时间2 用于处理嘴巴
start3 = np.arange(10)#时间3 用于处理点头
end1 =  np.arange(10)
end2 =  np.arange(10)
end3 =  np.arange(10)

start4 = 0
end4 = 0

flag1=np.arange(10)
flag3=np.arange(10)
eye_close_count = np.arange(10)
state = np.arange(10) #表示状态 0是开始 1是清醒 2是可能犯困 3是犯困


# cap.isOpened() 返回 true/false 检查初始化是否成功
while cap.isOpened():

    # cap.read()
    # 返回两个值：
    #    一个布尔值 true/false，用来判断读取视频是否成功/是否到视频末尾
    #    图像对象，图像的三维矩阵
    flag, im_rd = cap.read()
    
    # 待会要写的字体
    font = cv2.FONT_HERSHEY_SIMPLEX

    # 每帧数据延时 1ms，延时为 0 读取的是静态帧
    k = cv2.waitKey(1)

    # 取灰度
    img_gray = cv2.cvtColor(im_rd, cv2.COLOR_RGB2GRAY)
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    cv2.putText(im_rd, "FPS:" + str(fps), (550,20), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    
    # 人脸数
    faces = detector(img_gray, 0)

    
    # 检测到人脸
    
    if len(faces) != 0:
        # 68个特征点
        for i in range(len(faces)):
            landmarks = np.matrix([[p.x, p.y] for p in predictor(im_rd, faces[i]).parts()])

            # 标68个点
            for idx, point in enumerate(landmarks):
                
                    
                # 68点的坐标
                pos = (point[0, 0], point[0, 1])
                if idx == 0:
                    cv2.putText(im_rd, "face:"+str(i+1), pos, font, 0.5, (255, 0, 255), 1, cv2.LINE_AA)
                    #cv2.putText(im_rd, str(point[0, 0]), (20, 160), font, 0.5, (255, 0, 255), 1, cv2.LINE_AA)
                    #posleft = (point[0, 0], point[0, 1])
                
                # 特征点画圈
                #cv2.circle(im_rd, pos, 2, color=(255, 255, 255))
                # 写1-68
               # cv2.putText(im_rd, str(idx + 1), pos, font, 0.2, (187, 255, 255), 1, cv2.LINE_AA)
                
           
        for index,face in enumerate(faces):
            landmarks = predictor(img_gray, face)
            
            # 眼睛识别
            left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
            right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
            
            blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
            for num in range(0,14):
                eye_arr[num] = eye_arr[num+1]
        
            eye_arr[14] = blinking_ratio
            blinking_ratio= np.mean(eye_arr)   
                
            #cv2.putText(im_rd, str(blinking_ratio), (20+index*100, 60), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
            
            end1[index] = time.time()#记时，判断闭眼时间

            #检测眼睛状况
            if blinking_ratio > 5.5:
                cv2.putText(im_rd, " ", (20+index*200, 90), font, 0.8, (255, 255, 255)) #方法，作用：在图像上打印文字，设置字体，颜色，大小
                if flag1[index] != 2:
                    flag1[index] = 1
            else :
                cv2.putText(im_rd, " ", (20+index*200, 90), font, 0.8, (255, 255, 255))
                flag1[index] = 0
                flag3[index] = 1
                start1[index] = time.time()#记时
            #print("闭眼时间:%.2f秒"%(end-start))#获取睁闭眼时间差
            
            if (end1[index]-start1[index]) > 2 :
                cv2.putText(im_rd, " ", (20+index*200, 120), font, 0.8, (0, 0, 255))
                #state = 2
                flag1[index]= 2
               
            
            if flag1[index] == 2:
                if flag3[index] == 1:
                    eye_close_count[index] = eye_close_count[index] + 1
                    flag3[index] = 0
           
            if eye_close_count[index]==3:
                
                #start4 = time.time()
                state[index] = 3
            
        
            else:
                if flag1[index] == 0:
                    state[index] = 1
                else:
                    state[index] = 2
                
                    
            #嘴巴识别
            
            
            yawn_ratio = get_yawning_ratio([51, 53, 59, 57], landmarks)
                
            #cv2.putText(im_rd, str(yawn_ratio), (20+index*200, 330), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
            end2[index] = time.time()
            if yawn_ratio < 0.55:
                cv2.putText(im_rd, " ", (20+index*200, 360), font, 0.8, (255, 255, 255))
   
            else :
                cv2.putText(im_rd, " ", (20+index*200, 360), font, 0.8, (255, 255, 255))
                start2[index] = time.time()
                
            if (end2[index] - start2[index]) > 1.5 :
                cv2.putText(im_rd, " ", (20, 390), font, 0.8, (0, 0, 255))
                if (end1[index]-start1[index]) > 2 :
                    state[index] = 3
                else:
                    state[index] = 2
            #鼻子判断是不是在点头
            nod = noseposition([30], landmarks)
           # cv2.putText(im_rd, str(nod), (20+index*200, 420), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
            end3[index] = time.time()
            
            if nod > 700:
                cv2.putText(im_rd, " ", (20+index*200, 450), font, 0.8, (255, 255, 255))
                start3[index] = time.time()
                
            
                if (end1[index]-start1[index]) > 2 and (end3[index]-start3[index]) < 3 :
                    state[index] = 3
            #      cv2.putText(im_rd, "absolutely sleepy!", (20+index*200, 150), font, 0.8, (0, 0, 255))
            #        
            
            else :
                cv2.putText(im_rd," ", (20+index*200, 450), font, 0.8, (255, 255, 255))
            
            ######## 
            
            if state[index] == 1:
                drawframe([0,8,16,19,24], landmarks, 255, 0, 0)
            elif state[index] == 2:
                 drawframe([0,8,16,19,24], landmarks, 0, 255, 255)
            elif state[index] == 3:
                 drawframe([0,8,16,19,24], landmarks, 0, 0, 255)
            
            
            #######
        cv2.putText(im_rd, "Faces: " + str(len(faces)), (20, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        
    else:
        # 没有检测到人脸
        cv2.putText(im_rd, "No face detected", (20, 50), font, 1, (0, 0, 0), 1, cv2.LINE_AA)

    # 添加说明
    #im_rd = cv2.putText(im_rd, "'S': screenshot", (20, 460), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
    #im_rd = cv2.putText(im_rd, "esc: quit", (20, 480), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

    # 按下 's' 键保存
    if k == ord('s'):
        ss_cnt += 1
        print(path_screenshots + "ss_" + str(ss_cnt) + "_" +
              time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) + ".jpg")
        cv2.imwrite(path_screenshots + "ss_" + str(ss_cnt) + "_" +
                    time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) + ".jpg", im_rd)
    
    # 退出
    #key = cv2.waitKey(1)
    if k == 27:
        break

    # 窗口显示
    # 参数取 0 可以拖动缩放窗口，为 1 不可以
    cv2.namedWindow("camera", 0)
    #cv2.namedWindow("camera", 1)

    cv2.imshow("camera", im_rd)

# 释放摄像头
cap.release()

# 删除建立的窗口
cv2.destroyAllWindows()
cap.release()

D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_1_2019-10-28-18-50-01.jpg
D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_2_2019-10-28-18-50-23.jpg
D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_3_2019-10-28-18-51-17.jpg
D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_4_2019-10-28-18-53-40.jpg
D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_5_2019-10-28-18-53-53.jpg
D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_6_2019-10-28-18-54-01.jpg
D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_7_2019-10-28-18-54-02.jpg
D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_8_2019-10-28-18-54-04.jpg
D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_9_2019-10-28-18-54-49.jpg
D:/GitHub/2019-HMI-ISP-07/demo2/data/screenshots/ss_10_2019-10-28-18-54-51.jpg
